In [7]:
#
# Created on Fri Nov 24 2023 00:57:09
# Author: Mukai (Tom Notch) Yu
# Email: myual@connect.ust.hk
# Affiliation: Hong Kong University of Science and Technology
#
# Copyright Ⓒ 2023 Mukai (Tom Notch) Yu
#
import cv2
import numpy as np
import torch
from scipy.spatial.distance import cdist
from torch.utils.data import Dataset

from utils.files import read_file, print_dict

In [8]:
config = read_file("./configs/default.yaml")
print_dict(config)

train: 
  batch_size: 32.0
  epochs: 10.0
  learning_rate: 0.0001
  dataset: 
    COCO: 
      path: ./COCO2014/train2014
      fraction: 0.001
      resize: [640.0, 480.0]
      shuffle: true
    ScanNet: 
      path: ./ScanNet/train
      fraction: 0.001
      resize: [640.0, 480.0]
      shuffle: true
eval: 
  output_dir: ./dump_match_pairs/
  batch_size: 32.0
  eval_interval: 5.0
  dataset: 
    COCO: 
      path: ./COCO2014/eval2014
      fraction: 0.001
      resize: [640.0, 480.0]
      shuffle: true
    ScanNet: 
      path: ./ScanNet/eval
      fraction: 0.001
      resize: [640.0, 480.0]
      shuffle: true
superglue: 
  num_layers: 3.0
  sinkhorn_iterations: 10.0
  match_threshold: 0.2
  descriptor_dim: 256.0
feature_extraction: 
  max_keypoints: 64.0
  descriptor_dim: 256.0
  extractor: 
    SIFT: 
      contrastThreshold: 0.04
      edgeThreshold: 10.0
      sigma: 1.6
    superpoint: 
      keypoint_threshold: 0.005
      nms_radius: 4.0
      match_threshold: 0.2
      r

In [9]:
class COCOTrainingDataset(Dataset):
    def __init__(self, config: dict):
        self.config = config
        self.path = config["path"]
        self.fraction = config["fraction"]
        self.resize = config["resize"]
        self.shuffle = config["shuffle"]

In [10]:
coco_training_dataset = COCOTrainingDataset(config["train"]["dataset"]["COCO"])